In [1]:
import torch
import numpy as np
from scipy.stats import spearmanr, pearsonr
from scipy.signal import correlate

In [3]:
x = np.sin(np.arange(0, 10, 0.1))
y = np.cos(np.arange(0, 10, 0.1))

x = torch.from_numpy(x).float()
y = torch.from_numpy(y).float()

In [4]:
x_fft = torch.fft.rfft(x)
y_fft = torch.fft.rfft(y)

In [5]:
res = x_fft * torch.conj(y_fft) # x(f)x y*(f)
corr = torch.fft.irfft(res,n=x.shape[0],dim=-1)

In [6]:
sum(corr)/len(corr)

tensor(-0.8421)

In [40]:
np.corrcoef(x.numpy(), y.numpy())

array([[1., 1.],
       [1., 1.]])

In [9]:
t = torch.tensor([[1,2],[3,4]])
t

tensor([[1, 2],
        [3, 4]])

In [10]:
torch.conj(t)

tensor([[1, 2],
        [3, 4]])

In [11]:
X = torch.tensor([[1,2],[3,4]])
Y = torch.tensor([[2,1],[4,3]])

In [16]:
X_f = torch.fft.rfft(X)
Y_f = torch.fft.rfft(Y)
corr = torch.fft.irfft(X_f * torch.conj(Y_f), n=X.shape[0], dim=-1)

In [17]:
corr

tensor([[ 4.,  5.],
        [24., 25.]])

In [20]:
torch.mean(corr,dim=0)

tensor([14., 15.])

In [14]:
X_f * torch.conj(Y_f)

tensor([[ 9.+0.j, -1.+0.j],
        [49.+0.j, -1.+0.j]])

In [15]:
X_f * Y_f

tensor([[ 9.+0.j, -1.+0.j],
        [49.+0.j, -1.+0.j]])

In [ ]:
# 使用fft计算x, y 的相关性
# 1. fft(x) fft(y)
# 2. x(f)x y*(f)
# 3. ifft(x(f)x y*(f))
# 4. sum(ifft(x(f)x y*(f)))/len(ifft(x(f)x y*(f)))


In [62]:
x1 = torch.tensor([[1,2,3,4,5],[3,4,5,1,2]]) # [[x1_fft][x2_fft]]
x2 = torch.tensor([[3,4,5,6,7],[2,3,8,9,10]])
x1_f = torch.fft.rfft(x1, dim=-1)
x2_f = torch.fft.rfft(x1, dim=-1)
print(x1_f)

tensor([[ 1.5000e+01+0.0000j, -2.5000e+00+3.4410j, -2.5000e+00+0.8123j],
        [ 1.5000e+01+0.0000j,  1.1921e-07-4.2533j,  0.0000e+00+2.6287j]])


In [63]:
torch.fft.rfft(x1[0,:])

tensor([15.0000+0.0000j, -2.5000+3.4410j, -2.5000+0.8123j])

In [65]:
torch.fft.irfft(
    torch.tensor([1.5000e+01+0.0000j,  1.1921e-07-4.2533j,  0.0000e+00+2.6287j]),n=5)

tensor([3.0000, 4.0000, 5.0000, 1.0000, 2.0000])

In [75]:
x1_f@torch.conj(x2_f).T

tensor([[250.0000-2.2347e-07j, 212.5000-4.0615e+00j],
        [212.5000+4.0615e+00j, 250.0000+0.0000e+00j]])

In [77]:
a =x1_f[0] * torch.conj(x2_f[1])
torch.fft.irfft(a, n=x1.shape[1], dim=-1)

tensor([40., 45., 55., 45., 40.])

In [70]:
res = x1_f * torch.conj(x2_f)
print('res: ', res)
corr = torch.fft.irfft(res, n=x1.shape[1], dim=-1)
print('corr: ', corr)

res:  tensor([[225.0000+0.j,  18.0902+0.j,   6.9098+0.j],
        [225.0000+0.j,  18.0902+0.j,   6.9098+0.j]])
corr:  tensor([[55.0000, 45.0000, 40.0000, 40.0000, 45.0000],
        [55.0000, 45.0000, 40.0000, 40.0000, 45.0000]])


In [78]:
x2

tensor([[ 3,  4,  5,  6,  7],
        [ 2,  3,  8,  9, 10]])

In [84]:
# x2 -> float 32
x2 = x2.float()

In [86]:
torch.mean(x2, dim=0)

tensor([2.5000, 3.5000, 6.5000, 7.5000, 8.5000])

: 

### select children node

In [1]:
import torch

In [81]:
a = torch.tensor([
    [1,0,0],
    [1,1,1],
    [1,0,1]
])
x = torch.tensor([
    [[1,2,3]],
    [[0,3,7]],
    [[3,0,6]]
])
res = (a*x.T).permute(1,0,2)

In [85]:
x[2]

tensor([[3, 0, 6]])

In [82]:
res

tensor([[[1, 0, 0],
         [2, 0, 0],
         [3, 0, 0]],

        [[1, 0, 3],
         [2, 3, 0],
         [3, 7, 6]],

        [[1, 0, 3],
         [2, 0, 0],
         [3, 0, 6]]])

In [83]:
res[0]

tensor([[1, 0, 0],
        [2, 0, 0],
        [3, 0, 0]])

In [113]:
B, N, T = 2, 5, 3

def generate_ad():
    rand_mat = torch.rand(N, N)
    ad = (rand_mat > 0.5).float()
    # diag item set 0
    for i in range(N):
        ad[i, i] = 0
    return ad
        
def get_data():
    ''' generate node data, shape: (B * N * T)'''
    # set seed
    torch.manual_seed(0)
    return torch.randn(B, N, T)
data = get_data()
ad = generate_ad()
# data

In [122]:
def get_child(X, A):
    ''' x: (N, T), a: (N, N)'''
    return (A*X[0]).permute(1,0,2)

In [132]:
data[0].s

tensor([[-1.1258, -1.1524, -0.2506],
        [-0.4339,  0.8487,  0.6920],
        [-0.3160, -2.1152,  0.3223],
        [-1.2633,  0.3500,  0.3081],
        [ 0.1198,  1.2377, -0.1435]])

In [133]:
(ad*data[0].squeeze().T)

RuntimeError: The size of tensor a (5) must match the size of tensor b (3) at non-singleton dimension 0

In [126]:
data

tensor([[[-1.1258, -1.1524, -0.2506],
         [-0.4339,  0.8487,  0.6920],
         [-0.3160, -2.1152,  0.3223],
         [-1.2633,  0.3500,  0.3081],
         [ 0.1198,  1.2377, -0.1435]],

        [[-0.1116, -0.6136,  0.0316],
         [-0.4927,  0.2484,  0.4397],
         [ 0.1124, -0.8411, -2.3160],
         [-0.1023,  0.7924, -0.2897],
         [ 0.0525,  0.5229,  2.3022]]])

In [128]:
ad

tensor([[0., 1., 0., 0., 1.],
        [1., 0., 1., 1., 0.],
        [1., 0., 0., 0., 1.],
        [1., 0., 1., 0., 0.],
        [1., 1., 1., 1., 0.]])

In [125]:
print('data: ', data)
print('ad: ', ad)
child = get_child(data, ad)
print('child: ', child)

data:  tensor([[[-1.1258, -1.1524, -0.2506],
         [-0.4339,  0.8487,  0.6920],
         [-0.3160, -2.1152,  0.3223],
         [-1.2633,  0.3500,  0.3081],
         [ 0.1198,  1.2377, -0.1435]],

        [[-0.1116, -0.6136,  0.0316],
         [-0.4927,  0.2484,  0.4397],
         [ 0.1124, -0.8411, -2.3160],
         [-0.1023,  0.7924, -0.2897],
         [ 0.0525,  0.5229,  2.3022]]])
ad:  tensor([[0., 1., 0., 0., 1.],
        [1., 0., 1., 1., 0.],
        [1., 0., 0., 0., 1.],
        [1., 0., 1., 0., 0.],
        [1., 1., 1., 1., 0.]])


RuntimeError: The size of tensor a (5) must match the size of tensor b (3) at non-singleton dimension 1

In [45]:
import torch
import numpy as np
import scipy# .signal.correlate
import time

In [102]:
N = 5
T = 11
# 随机生成邻接矩阵
adj = np.random.randint(0, 2, (N, N))
X = torch.randn(N, T)

In [103]:
def time_rec(func):
    def wrap_func(*args, **kwargs):
        start = time.time()
        corr = func(*args, **kwargs)
        end = time.time()
        print('running time: ', end-start)
        return corr
    return wrap_func
        
@time_rec
def cal_corr(x_0, x_1, f):
    corr = f(x_0, x_1)
    return corr

In [136]:
time_delay = cal_corr(X[0], X[0].roll(3), scipy.signal.correlate)
print(time_delay)
max_lag = 5
print(-np.argmax(time_delay[len(time_delay)//2-max_lag:len(time_delay)])+len(time_delay)//2-max_lag)
print(X.shape[-1] - np.argmax(time_delay)-1)

running time:  0.0
[ 1.2468162   3.2639039  -0.9340196  -0.4098159   4.262111   -1.5268508
 -0.31321257  9.651836   -0.21992397 -4.8084626   1.2743511   1.4615388
 -2.1905522   2.007371    3.118171   -2.98776    -3.5425339   0.31444553
  1.5643156   0.22115692 -0.26092228]
3
3


In [140]:
np.roll(np.diag(adj),1)

array([1, 1, 0, 0, 1])

In [5]:
import torch
import torch.nn

In [6]:
x = torch.randn(5,3,8,16) # B * C * N * T
conv1 = torch.nn.Conv1d(3, 8, 3, stride=1)
conv1(x)

RuntimeError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [5, 3, 8, 16]

In [7]:
x = torch.randn(2,3,3)
x

tensor([[[-1.2050,  0.2151, -1.7792],
         [-0.7125,  1.5173,  0.7709],
         [ 0.6298,  0.0029,  1.3107]],

        [[-0.9602, -0.1017, -0.7714],
         [ 0.7244,  2.2775,  0.1156],
         [ 0.8581,  0.1368,  0.4027]]])

In [19]:
x_d = torch.diagonal(x, dim1=-2, dim2=-1)
x_d

tensor([[-1.2050,  1.5173,  1.3107],
        [-0.9602,  2.2775,  0.4027]])

In [20]:
torch.diag_embed(x_d)

tensor([[[-1.2050,  0.0000,  0.0000],
         [ 0.0000,  1.5173,  0.0000],
         [ 0.0000,  0.0000,  1.3107]],

        [[-0.9602,  0.0000,  0.0000],
         [ 0.0000,  2.2775,  0.0000],
         [ 0.0000,  0.0000,  0.4027]]])

In [1]:
import argparse

In [7]:

def get_args():
    parser = argparse.ArgumentParser(description='Training parameters! ')

    ###########################################
    #############         Common parameters:
    ###########################################
    parser.add_argument('--results_path', type=str, required=False, default='./results/')
    parser.add_argument('--data_path', type=str, required=False, default='./data/save_df.csv', help='data path') 
    # parser.add_argument('--data_path', type=str, required=False, default='./data/te_data.csv', help='data path')  # te dataset
    
    args = parser.parse_args()
    return args

In [9]:
d = {'results_path': './results/', 'data_path': './data/save_df.csv', 'rolling_window': 1, 'scaler': 'StandardScaler', 'train_ratio': 0.6, 'valid_ratio': 0.1, 'time': 'day', 'seed': 42, 'epoches': 10, 'lr': 0.001, 'WEIGHT_DECAY': 0.001, 'batch_size': 16, 'model': 'gru', 'target': '汽油', 'window_size': 12, 'autoregress': False, 'lstm_layers': 3, 'lstm_hidden_size': 256, 'hidde_gat': 32}

In [15]:
d['data_path']

'./data/save_df.csv'

In [13]:
def dict_arg_f(test=None,**args):
    print(test)
    print(args)
    
dict_arg_f(**d)

None
{'results_path': './results/', 'data_path': './data/save_df.csv', 'rolling_window': 1, 'scaler': 'StandardScaler', 'train_ratio': 0.6, 'valid_ratio': 0.1, 'time': 'day', 'seed': 42, 'epoches': 10, 'lr': 0.001, 'WEIGHT_DECAY': 0.001, 'batch_size': 16, 'model': 'gru', 'target': '汽油', 'window_size': 12, 'autoregress': False, 'lstm_layers': 3, 'lstm_hidden_size': 256, 'hidde_gat': 32}


In [1]:
import pandas as pd

In [21]:
x = pd.DataFrame({'a': [1,2,3], 'b': [4,5,6]})
x['a'].shift(1)

0    NaN
1    1.0
2    2.0
Name: a, dtype: float64

In [19]:
path = r'G:\Code\FrameWork_version_1.0_gat\results\20240112155519\GAT\record.txt'
parameter_name='window_size'
with open(path, 'r') as f:
    lines = f.readlines()[-1]
    args = lines.strip().split(',')
    print(args)
    para_value = [arg.split('=')[1] for arg in args if arg.split('=')[0].strip() == parameter_name][0]
    seed = [arg.split('=')[1] for arg in args if arg.split('=')[0].strip() == 'seed'][0]
    print(para_value, seed)
    

["Namespace(results_path='./results/'", " data_path='./data/te_data.csv'", " dataset='TE'", " target='v10'", " rmcols='None'", " time='20240112155519'", ' rolling_window=1', " scaler='MinMaxScaler'", ' train_ratio=0.6', ' valid_ratio=0.1', ' seed=30', " optimizer='Adam'", ' epoches=5', ' lr=0.001', ' WEIGHT_DECAY=0.0001', ' batch_size=16', " device='cuda'", " model_type='GAT'", ' window_size=32', ' autoregress=False', ' lstm_layers=3', ' lstm_hidden_size=256', " graph_type='mechanism'", ' nhid=32', ' target_idx=9', ' attgru_hidsize=128', ' attgru_hidslayer=32', ' gru_hid=32', ' node_num=33)']
32 30


In [35]:
path = r'G:\Code\FrameWork_version_1.0_gat\results\20240112163004\GAT\record.txt'
parameter_name='window_size'
with open(path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        print(line)
print(lines[1].split(':')[1].strip())

Target: v10

Rmse: 0.11987720149935367

mse: 0.014370543439316643

mae: 0.0980003526271062

mape: 3.3694284067505644

R2: 0.18935821565135103

params: 

Namespace(results_path='./results/', data_path='./data/te_data.csv', dataset='TE', target='v10', rmcols='None', time='20240112163004', rolling_window=1, scaler='MinMaxScaler', train_ratio=0.6, valid_ratio=0.1, seed=128, optimizer='Adam', epoches=5, lr=0.001, WEIGHT_DECAY=0.0001, batch_size=16, device='cuda', model_type='GAT', window_size=32, autoregress=False, lstm_layers=3, lstm_hidden_size=256, graph_type='mechanism', nhid=32, target_idx=9, attgru_hidsize=128, attgru_hidslayer=32, gru_hid=32, node_num=33)

0.11987720149935367


In [23]:
x = pd.DataFrame(columns=['a', 'b'])
x

,a,b


In [24]:
data = [1,2]
x = pd.concat([x, pd.DataFrame([data], columns=['a', 'b'])])

In [25]:
x

,a,b
0,1,2


In [36]:
float('1')

1.0